In [1]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [5]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PRODUCT_ID  225563 non-null  object
 1   ORDER_DATE  225563 non-null  object
 2   ORDER_QTY   225563 non-null  int16 
dtypes: int16(1), object(2)
memory usage: 3.9+ MB


In [7]:
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [8]:
data

,PRODUCT_ID,ORDER_DATE,ORDER_QTY
0,Product_001,2019-01-02,850
1,Product_004,2019-01-02,700
2,Product_004,2019-01-02,900
3,Product_004,2019-01-02,550
4,Product_004,2019-01-02,300
...,...,...,...
225558,Product_098,2024-08-31,200
225559,Product_100,2024-08-31,800
225560,Product_100,2024-08-31,550
225561,Product_100,2024-08-31,850


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_DATE  225563 non-null  datetime64[ns]
 2   ORDER_QTY   225563 non-null  int16         
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB


In [10]:
# Group data by 'product_id'
grouped_data = data.groupby('PRODUCT_ID')

In [11]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['product_id','order_date', 'test_forecast_orders','rse', 'rmse','r2_score'])

In [12]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['product_id','order_date', 'forecast_order_qty'])

In [13]:
grouped_data

In [ ]:
for product_id, group in grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]
    
    # Create SARIMAX model
    model = SARIMAX(train_data['ORDER_QTY'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    # Evaluate the model
    rse = np.sqrt(((test_df - test_data) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], test_df))
    r2 = r2_score(test_data['ORDER_QTY'], test_df)

    # Convert index to date format
    test_df.index = pd.to_datetime(test_df.index)
    date_list = test_df.index.strftime('%Y-%m-%d').to_list()
    
    new_row = {
        'product_id': product_id,
        'order_date': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)
    test_results = pd.concat([test_results, df_new], ignore_index=True)
    
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)
    forecast_df = forecast.predicted_mean
    
    # Convert index to date format
    forecast_df.index = pd.to_datetime(forecast_df.index)
    date_list2 = forecast_df.index.strftime('%Y-%m-%d').to_list()
    
    new_row2 = {
        'product_id': product_id,
        'order_date': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    forecast_results = pd.concat([forecast_results, df_new2], ignore_index=True)
